In [43]:

import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT # <-- ADD THIS LINE
from psycopg2.extensions import AsIs, QuotedString
import pandas as pd
import psycopg2.extras
import json
import openai
import os
import yaml


In [44]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

openai.api_key  = PARAM['openai_api']
client = openai.OpenAI(api_key = PARAM['openai_api'])

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [45]:
username = "age"
password = "AgeVector"
host = "localhost"
db_name = 'drug'
graph_name = "drug_disorder"



In [46]:
## create a database



con = psycopg2.connect(dbname="postgres",
      user=username, host=host,
      password=password)

con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT) # <-- ADD THIS LINE

cur = con.cursor()

cur.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = '{db_name}';")

exists = cur.fetchone()

if not exists:
    cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
    print ("created database")
else:
    print ("database already exists")
    
cur.close()


created database


In [47]:


connection = psycopg2.connect(database=db_name, user=username, password=password, host=host, port=5432)

cur = connection.cursor()

cur.execute("SELECT current_database();")
print (cur.fetchone()[0])


drug


In [48]:
cur.execute("""
                CREATE EXTENSION IF NOT EXISTS age;
                LOAD 'age';
                SET search_path = ag_catalog, "$user", public;
               """)

cur.execute("""
                CREATE EXTENSION vector;
               """)

In [49]:

cur.execute("""CREATE TABLE IF NOT EXISTS Trials (
            PostingID     INT NOT NULL PRIMARY KEY,
            Sponsor    VARCHAR(1024),
            StudyTitle     VARCHAR(2048),
            Drug   VARCHAR(1024),
            Disorder   VARCHAR(1024),
            Phase   VARCHAR(512),
            LinkToSponsorStudyRegistry   VARCHAR(2048),
            LinkToClinicalTrials   VARCHAR(2048),
            cui  text[],
            PreferredUMLSName  text[],
            StudyTitle_Tokens tsvector GENERATED ALWAYS AS (to_tsvector('english', StudyTitle)) STORED
            );""")


In [50]:
df = pd.read_csv("drugs/trials_umls.tsv", sep='\t', lineterminator='\n')
values = []

for index, row in df.iterrows():
    values.append((row["Posting ID"], row["Sponsor"], row["Study Title"], row["Medicine or Vaccine (generic name)"], row["Medical Condition"], row["Phase"], row["Link to Sponsor Study Registry"], row["Link to study details on ClinicalTrials.gov (if available)"], row["CUI_umls"].split(";"), row["preferred_umls_name"].split(";")))

print (values[:2])

[(1, 'GSK', 'U0289-401: An Evaluator Blinded, 8 Week, Split-Face Study to Evaluate and Compare the Efficacy and Tolerability of MAXCLARITY II and PROACTIV in Subjects with Acne', 'benzoyl peroxide/salicylic acid', 'Acne Vulgaris', 'Phase 4', 'https://www.gsk-studyregister.com/en/trial-details/?id=114550', 'http://clinicaltrials.gov/show/NCT01706250', ['C0005088', 'C0036079'], ['benzoyl peroxide', 'salicylic acid']), (3, 'GSK', 'GSK1550188, A randomised, single-blind, placebo controlled, dose ascending, single dose study to evaluate the safety, tolerability, pharmacokinetics, and pharmacodynamic of GSK1550188 in Japanese subjects with Systemic Lupus Erythematosus (SLE)', 'belimumab', 'Systemic Lupus Erythematosus', 'Phase 1', 'https://www.gsk-studyregister.com/en/trial-details/?id=114243', 'http://clinicaltrials.gov/show/NCT01381536', ['C1723401'], ['belimumab'])]


In [51]:

psycopg2.extras.execute_batch(cur, "INSERT INTO Trials (PostingID, Sponsor, StudyTitle, Drug, Disorder, Phase, LinkToSponsorStudyRegistry, LinkToClinicalTrials, cui, PreferredUMLSName) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", values)

query = sql.SQL("CREATE INDEX ts_idx ON Trials USING GIN (StudyTitle_Tokens);")


In [52]:
cur.execute("""SELECT * FROM Trials LIMIT 5;""")

record = cur.fetchall()

for r in record:
    print(r)

(1, 'GSK', 'U0289-401: An Evaluator Blinded, 8 Week, Split-Face Study to Evaluate and Compare the Efficacy and Tolerability of MAXCLARITY II and PROACTIV in Subjects with Acne', 'benzoyl peroxide/salicylic acid', 'Acne Vulgaris', 'Phase 4', 'https://www.gsk-studyregister.com/en/trial-details/?id=114550', 'http://clinicaltrials.gov/show/NCT01706250', ['C0005088', 'C0036079'], ['benzoyl peroxide', 'salicylic acid'], "'-401':2 '8':6 'acn':28 'blind':5 'compar':15 'efficaci':17 'evalu':4,13 'face':10 'ii':22 'maxclar':21 'proactiv':24 'split':9 'split-fac':8 'studi':11 'subject':26 'toler':19 'u0289':1 'week':7")
(3, 'GSK', 'GSK1550188, A randomised, single-blind, placebo controlled, dose ascending, single dose study to evaluate the safety, tolerability, pharmacokinetics, and pharmacodynamic of GSK1550188 in Japanese subjects with Systemic Lupus Erythematosus (SLE)', 'belimumab', 'Systemic Lupus Erythematosus', 'Phase 1', 'https://www.gsk-studyregister.com/en/trial-details/?id=114243', 'ht

In [53]:
cur.execute("""SELECT COUNT(*) FROM Trials;""")

record = cur.fetchall()

for r in record:
    print(r)

(2179,)


In [54]:
#cur.execute(f"SELECT create_graph('{graph_name}');")

query = sql.SQL("SELECT create_graph('drug_disorder');")
cur.execute(query)



In [55]:
# query = sql.SQL("select {field} from {table} where {pkey} = %s").format(
#     field=sql.Identifier('my_name'),
#     table=sql.Identifier('some_table'),
#     pkey=sql.Identifier('id'))

In [56]:
cur.execute("""CREATE TABLE IF NOT EXISTS Disorder_vector (
            cui     VARCHAR(1024),
            definitionEmbedding  vector(1536)
            );""")

In [57]:

for line in open("drugs/disease_4_embedding.json", "r"):
    value = json.loads(line)
    #print (value["cui"])
    d = ""
    if "definition" not in value or value["definition"] is None:
        d = ""
    else:
        d = value["definition"].replace('"', "'")  

    #cur.execute("SELECT * FROM cypher(%s, $$MERGE (:Disorder {cui: %s, name: %s, definition: %s}) $$) as (n agtype);", (graph_name, QuotedString(value["cui"]), QuotedString(value.get("name", "")).getquoted(), QuotedString(d)))
    # query = f"""SELECT * FROM cypher('{graph_name}', 
    # $$MERGE (:Disorder {{cui: {value["cui"].replace('"', '')}, name: {value.get("name", "").replace('"', '').replace("'", "''")}, definition: {d}}}) $$)
    # as (n agtype);"""

    
    query = sql.SQL("SELECT * FROM cypher('drug_disorder', $$MERGE (:Disorder {{cui: {cui}, name: {name}, definition: {definition} }}) $$) as (n agtype);").format(
        cui = sql.Identifier(value["cui"]),
        name = sql.Identifier(value["name"]),
        definition = sql.Identifier(d)
    )
    cur.execute(query)

    if "definitionEmbedding"  in value and len(value["definitionEmbedding"]) > 0:
        cur.execute("INSERT INTO Disorder_vector (cui, definitionEmbedding) VALUES (%s,%s)", (value["cui"], value["definitionEmbedding"]))


    #query = "SELECT * FROM cypher('drug_disorder', $$MERGE (:Disorder {cui: %s, name: %s, definition: %s}) $$) as (n agtype);"

    #cur.execute(query, (value["cui"], value["name"].replace("'", "\\'"), d))

    #psycopg2.extras.execute_batch(cur, "SELECT * FROM cypher('drug_disorder', $$MERGE (:Disorder {cui: %s, name: %s, definition: %s }) $$) as (n agtype);", [value["cui"], value["name"], d])


#query = f"""SELECT * FROM cypher('{graph_name}', 
#    $$CREATE CONSTRAINT FOR (n:Disorder) REQUIRE n.cui IS UNIQUE $$)
#    as (n agtype);"""


#cur.execute(query)


In [58]:
for line in open("drugs/drugs_4.json", "r"):
    value = json.loads(line)
    #print (value)
    # query = f"""SELECT * FROM cypher('{graph_name}', 
    # $$MERGE (:Drug {{cui: {value["cui"].replace('"', '')}, name: {value.get("name", "").replace('"', '').replace("'", "''")}}}) $$)
    # as (n agtype);"""

    query = sql.SQL("SELECT * FROM cypher('drug_disorder', $$MERGE (:Drug {{cui: {cui}, name: {name} }}) $$) as (n agtype);").format(
        cui = sql.Identifier(value["cui"]),
        name = sql.Identifier(value["name"]),
    )
    cur.execute(query)

# query = f"""SELECT * FROM cypher('{graph_name}', 
#     $$CREATE CONSTRAINT ON (n:Drug) ASSERT n.cui IS UNIQUE; $$)
#     as (n agtype);"""

# cur.execute(query)

In [59]:
df = pd.read_csv("drugs/moa_3.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():
    #values.append((row["id"], row["name"]))

    # query = f"""SELECT * FROM cypher('{graph_name}', 
    # $$MERGE (:MOA {{moa_id: '{row["id"].replace('"', '')}', name: \"{row["name"].replace('"', '')}\"}}) $$)
    # as (n agtype);"""

    query = sql.SQL("SELECT * FROM cypher('drug_disorder', $$MERGE (:MOA {{moa_id: {moa_id}, name: {name} }}) $$) as (n agtype);").format(
        moa_id = sql.Identifier(row["id"]),
        name = sql.Identifier(row["name"]),
    )
    cur.execute(query)

# query = f"""SELECT * FROM cypher('{graph_name}', 
#     $$CREATE CONSTRAINT ON (n:MOA) ASSERT n.moa_id IS UNIQUE; $$)
#     as (n agtype);"""

# cur.execute(query)

In [60]:
df = pd.read_csv("drugs/cui_indication_3.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():
    #values.append((row["from"], row["to"]))

    # query = f"""
    #     SELECT * FROM cypher('{graph_name}', $$ MERGE (a:Drug {{cui: '{row["from"].replace('"', '')}'}}) MERGE (b:Disorder {{cui: '{row["to"].replace('"', '')}'}}) MERGE (a)-[e:MAY_TREAT]->(b) RETURN e $$) as (e agtype);
    # """

    query = sql.SQL("SELECT * FROM cypher('drug_disorder', $$MERGE (a:Drug {{cui: {from_id} }}) MERGE (b:Disorder {{cui: {to_id}}}) MERGE (a)-[e:MAY_TREAT]->(b) RETURN e $$) as (e agtype);").format(
        from_id = sql.Identifier(row["from"]),
        to_id = sql.Identifier(row["to"]),
    )

    cur.execute(query)

In [61]:
df = pd.read_csv("drugs/cui_moa_3.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():
    #values.append((row["from"], row["to"]))

    # query = f"""
    #     SELECT * FROM cypher('{graph_name}', $$ MERGE (a:Drug {{cui: '{row["from"].replace('"', '')}'}}) MERGE (b:MOA {{moa_id: '{row["to"].replace('"', '')}'}}) MERGE (a)-[e:HAS_MOA]->(b) RETURN e $$) as (e agtype);
    # """

    query = sql.SQL("SELECT * FROM cypher('drug_disorder', $$MERGE (a:Drug {{cui: {from_id} }}) MERGE (b:MOA {{moa_id: {to_id}}}) MERGE (a)-[e:HAS_MOA]->(b) RETURN e $$) as (e agtype);").format(
        from_id = sql.Identifier(row["from"]),
        to_id = sql.Identifier(row["to"]),
    )

    cur.execute(query)

In [62]:
#psycopg2.extras.execute_batch(cur, "INSERT INTO trials (PostingID, Sponsor, StudyTitle, Drug, Disorder, Phase, LinkToSponsorStudyRegistry, LinkToClinicalTrials, cui, PreferredUMLSName) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", values)


In [63]:
connection.commit()

In [64]:
cur.execute(f"""
SELECT * FROM cypher('{graph_name}', $$
        MATCH (V)-[R]-(V2)
        RETURN V,R,V2 LIMIT 5
$$) as (V agtype, R agtype, V2 agtype);
""")

record = cur.fetchall()

for r in record:
        print(r)

('{"id": 844424930131969, "label": "Disorder", "properties": {"cui": "C0023321", "name": "Lentigo", "definition": "Small circumscribed melanoses resembling, but differing histologically from, freckles. The concept includes senile lentigo (\'liver spots\') and nevoid lentigo (nevus spilus, lentigo simplex) and may also occur in association with multiple congenital defects or congenital syndromes (e.g., Peutz-Jeghers syndrome)."}}::vertex', '{"id": 1688849860263937, "label": "MAY_TREAT", "end_id": 844424930131969, "start_id": 1125899906842625, "properties": {}}::edge', '{"id": 1125899906842625, "label": "Drug", "properties": {"cui": "C0048318", "name": "mequinol"}}::vertex')
('{"id": 844424930131970, "label": "Disorder", "properties": {"cui": "C0021400", "name": "Influenza", "definition": "An acute viral infection in humans involving the respiratory tract. It is marked by inflammation of the NASAL MUCOSA; the PHARYNX; and conjunctiva, and by headache and severe, often generalized, myalgi

In [65]:
question = "joint-related disorders"

question_embedding = get_embedding(question)

#SELECT * FROM items ORDER BY embedding <-> '[3,1,2]' LIMIT 5;

cur.execute(f"""
SELECT cui FROM Disorder_vector ORDER BY definitionEmbedding <=> '{question_embedding}' LIMIT 5;
""")

record = cur.fetchall()

for r in record:
        print(r)

('C0022408',)
('C0949691',)
('C0026857',)
('C0035435',)
('C0152087',)


In [66]:
question = "joint-related disorders"

question_embedding = get_embedding(question)

#SELECT * FROM items ORDER BY embedding <-> '[3,1,2]' LIMIT 5;

# cur.execute(f"""
# WITH joint_related_disorders AS (
#     SELECT cui FROM disoder_vector ORDER BY definitionEmbedding <=> '{question_embedding}' LIMIT 5
# ), drug_for_disorder AS(
#     SELECT drug_name, drug_cui, disorder_name, disorder_cui FROM (SELECT * FROM cypher('drug_disorder', $$ MATCH (d:Drug)-[:MAY_TREAT]->(di:Disorder) RETURN DISTINCT d.cui, d.name, di.cui, di.name $$) as (drug_cui TEXT, drug_name TEXT, disorder_cui TEXT, disorder_name TEXT))
# )

# SELECT * FROM joint_related_disorders, drug_for_disorder WHERE joint_related_disorders.cui = drug_for_disorder.disorder_cui;
# """)

cur.execute(f"""
WITH joint_related_disorders AS (
    SELECT cui FROM Disorder_vector ORDER BY definitionEmbedding <=> '{question_embedding}' LIMIT 5
), drug_for_disorder AS(
    SELECT * FROM cypher('drug_disorder', $$ MATCH (d:Drug)-[:MAY_TREAT]->(di:Disorder) RETURN DISTINCT d.cui, d.name, di.cui, di.name $$) as (drug_cui TEXT, drug_name TEXT, disorder_cui TEXT, disorder_name TEXT)
)

SELECT * FROM joint_related_disorders, drug_for_disorder WHERE joint_related_disorders.cui = drug_for_disorder.disorder_cui;
""")

record = cur.fetchall()

for r in record:
        print(r)

('C0035435', 'C0063077', 'hydrocortisone acetate', 'C0035435', 'Rheumatism')
('C0035435', 'C0113286', 'dexamethasone sodium phosphate', 'C0035435', 'Rheumatism')
('C0035435', 'C0032950', 'prednisolone', 'C0035435', 'Rheumatism')
('C0035435', 'C0032952', 'prednisone', 'C0035435', 'Rheumatism')
('C0035435', 'C0030817', 'penicillamine', 'C0035435', 'Rheumatism')
('C0035435', 'C0040866', 'triamcinolone acetonide', 'C0035435', 'Rheumatism')
('C0035435', 'C0071839', 'prednisolone acetate', 'C0035435', 'Rheumatism')
('C0035435', 'C0025815', 'methylprednisolone', 'C0035435', 'Rheumatism')
('C0035435', 'C0063076', 'hydrocortisone buteprate', 'C0035435', 'Rheumatism')
('C0035435', 'C0020336', 'hydroxychloroquine', 'C0035435', 'Rheumatism')
('C0035435', 'C0010583', 'cyclophosphamide', 'C0035435', 'Rheumatism')
('C0035435', 'C0174938', 'methylprednisolone aceponate', 'C0035435', 'Rheumatism')
('C0035435', 'C0010137', 'cortisone', 'C0035435', 'Rheumatism')
('C0035435', 'C0011777', 'dexamethasone', 

In [67]:
connection.commit()

In [68]:
cur.execute(f"""
SELECT cui FROM trials LIMIT 3;
""")

record = cur.fetchall()

print("Data from Database:- ", record)

Data from Database:-  [(['C0005088', 'C0036079'],), (['C1723401'],), (['C0065767'],)]


In [69]:
cur.execute(f"""
SELECT cui FROM cypher('drug_disorder', $$
        MATCH (d:Drug) RETURN d.cui LIMIT 3
$$) as (cui text);
""")

record = cur.fetchall()

print("Data from Database:- ", record)

Data from Database:-  [('C0048318',), ('C0035629',), ('C0027603',)]


In [70]:
query = f"""
SELECT * FROM cypher('drug_disorder', $$
  MATCH (d:Drug)-[:MAY_TREAT]->(di:Disorder) 
  WHERE di.name = "Alzheimer's Disease" 
  RETURN DISTINCT d.cui, d.name
$$) as (drug_cui TEXT, drug_name TEXT);
"""

cur.execute(query)

record = cur.fetchall()

for r in record:
    print(r)

('C4043101', 'aducanumab')
('C0949312', 'galantamine hydrobromide')
('C0042291', 'valproic acid')
('C2697594', 'alpha-tocopherol succinate, D-')
('C0042874', 'tocopherol')
('C0078375', 'alpha-tocopherol succinate')
('C5242453', 'lecanemab')
('C1323100', 'tocopherol,DL-alpha')
('C0036579', 'selegiline')
('C0016967', 'galantamine')
('C0771848', 'donepezil hydrochloride')
('C0772415', 'rivastigmine tartrate')
('C0994472', 'dl-alpha tocopheryl acetate')
('C0527316', 'donepezil')
('C0025242', 'memantine')
('C0039245', 'tacrine')
('C0771988', 'memantine hydrochloride')
('C0649350', 'rivastigmine')
('C0078373', 'tocopherol acetate')
('C2697593', 'd-alpha-tocopheryl acetate')


In [71]:
# cur.execute("""
# SELECT t.StudyTitle FROM trials AS t WHERE (SELECT cui FROM cypher('drug_disorder', $$ MATCH (d:Drug)-[:MAY_TREAT]->(di:Disorder {name: "Alzheimer's Disease"}) RETURN d.cui $$) as (cui VARCHAR(25))) = ANY(t.cui);
# """)

# cur.execute("""
# SELECT t.StudyTitle FROM trials AS t, (SELECT cui FROM cypher('drug_disorder', $$ MATCH (d:Drug)-[:MAY_TREAT]->(di:Disorder {name: "Alzheimer''s Disease"}) RETURN DISTINCT d.dui $$) as (cui VARCHAR(25))) AS drugs WHERE drugs.cui = ANY(t.cui);
# """)

query = """
WITH drug_for_disorder AS(
SELECT * FROM cypher('drug_disorder', $$ MATCH (d:Drug)-[:MAY_TREAT]->(di:Disorder) WHERE di.name = "Alzheimer's Disease" RETURN DISTINCT d.cui, d.name $$) as (drug_cui TEXT, drug_name TEXT)
)
SELECT Trials.StudyTitle, drug_for_disorder.drug_name FROM Trials, drug_for_disorder WHERE drug_for_disorder.drug_cui = ANY(Trials.cui);

"""

cur.execute(query)



#works
# cur.execute("""
# SELECT t.StudyTitle FROM trials AS t WHERE 'C1948374' = ANY(t.cui);
# """)

record = cur.fetchall()

for r in record:
    print(r)

('A Multi-Center, Randomized, Double-Blind, Parallel Group Study With 3 Groups Receiving Placebo, 5 mg of E2020 and 10 mg of E2020', 'donepezil')
("Clinical efficacy and safety of donepezil on cognitive and global function in patients with Alzheimer's disease. A 24-week, multicenter, double-blind, placebo-controlled study in Japan.", 'donepezil')
("A Preliminary, Multi-Center, Randomized, Double-Blind, Placebo-Controlled Evaluation of the Safety and Efficacy of E2020 in Patients with Alzheimer's Disease.", 'donepezil')
("A 15-week, Multi-Center, Randomized, Double-Blind, Placebo-Controlled Evaluation of the Safety and Efficacy of E2020 in Patients with Alzheimer's Disease.", 'donepezil')
("A 30-Week, Multi-Center, Randomized, Double-Blind, Placebo-Controlled Evaluation of the Safety and Efficacy of E2020 in Patients with Alzheimer's Disease.", 'donepezil')
("A 30-Week, Multi-Center, Randomized, Double-Blind, Placebo-Controlled Evaluation of the Safety and Efficacy of E2020 in Patients 

In [72]:
query = """

SELECT Trials.StudyTitle, drug_name 
FROM Trials, cypher('drug_disorder', $$ MATCH (d:Drug)-[:MAY_TREAT]->(di:Disorder) WHERE di.name = "Alzheimer's Disease" RETURN DISTINCT d.cui, d.name $$) as (drug_cui TEXT, drug_name TEXT)
WHERE drug_cui = ANY(Trials.cui);

"""

cur.execute(query)



#works
# cur.execute("""
# SELECT t.StudyTitle FROM trials AS t WHERE 'C1948374' = ANY(t.cui);
# """)

record = cur.fetchall()

for r in record:
    print(r)

('A Multi-Center, Randomized, Double-Blind, Parallel Group Study With 3 Groups Receiving Placebo, 5 mg of E2020 and 10 mg of E2020', 'donepezil')
("Clinical efficacy and safety of donepezil on cognitive and global function in patients with Alzheimer's disease. A 24-week, multicenter, double-blind, placebo-controlled study in Japan.", 'donepezil')
("A Preliminary, Multi-Center, Randomized, Double-Blind, Placebo-Controlled Evaluation of the Safety and Efficacy of E2020 in Patients with Alzheimer's Disease.", 'donepezil')
("A 15-week, Multi-Center, Randomized, Double-Blind, Placebo-Controlled Evaluation of the Safety and Efficacy of E2020 in Patients with Alzheimer's Disease.", 'donepezil')
("A 30-Week, Multi-Center, Randomized, Double-Blind, Placebo-Controlled Evaluation of the Safety and Efficacy of E2020 in Patients with Alzheimer's Disease.", 'donepezil')
("A 30-Week, Multi-Center, Randomized, Double-Blind, Placebo-Controlled Evaluation of the Safety and Efficacy of E2020 in Patients 

In [73]:
cur.execute("""
SELECT StudyTitle, cui FROM Trials WHERE StudyTitle_Tokens @@ to_tsquery('english', 'double-blind & randomize & phase');
""")

record = cur.fetchall()

for r in record:
    print(r)

('A Multicenter, Randomized, Double-Blind, Phase 3 Study of the Safety and Efficacy of Emulsion Formulation U0267, versus Vehicle in Subjects with Plaque-type Psoriasis', ['C0065767'])
('A Phase 3 Multicenter, Randomized, Double-Blind, Active And Vehicle-Controlled Study Of The Safety And Efficacy Of A Clindamycin / Benzoyl Peroxide Gel Versus Clindamycin Gel Versus Benzoyl Peroxide Gel Versus Vehicle Gel In Subjects With Acne Vulgaris', ['C0005088', 'C0055881'])
('A Randomized, Double-Blind, Multicenter, Placebo-controlled, Phase III Superiority Study to Assess the Safety and Efficacy of Topical Retapamulin Ointment, 1%, versus Placebo Ointment Applied Twice Daily for 5 Days in the Treatment of Adult and Pediatric Subjects with SITL', ['C1703334'])
('A multicenter, randomized, 12 week, double-blind, placebo-controlled, parallel-group, Phase IIa study using 18F fluorodeoxyglucose (FDG)-PET to measure the effects of rilapladib on macrophage activity in subjects with atherosclerosis', ['

In [74]:
question = "joint-related disorders"

question_embedding = get_embedding(question)

cur.execute(f"""
WITH joint_related_disorders AS (
    SELECT cui FROM Disorder_vector ORDER BY definitionEmbedding <=> '{question_embedding}' LIMIT 10
), drug_for_disorder AS (
    SELECT * FROM cypher('drug_disorder', $$ MATCH (d:Drug)-[:MAY_TREAT]->(di:Disorder) RETURN DISTINCT d.cui, d.name, di.cui, di.name $$) as (drug_cui TEXT, drug_name TEXT, disorder_cui TEXT, disorder_name TEXT)
)

SELECT Trials.StudyTitle, drug_for_disorder.disorder_name , drug_for_disorder.drug_name FROM Trials, joint_related_disorders, drug_for_disorder WHERE joint_related_disorders.cui = drug_for_disorder.disorder_cui AND drug_for_disorder.drug_cui = ANY(Trials.cui);
""")

record = cur.fetchall()

for r in record:
        print(r)

('A Randomized, Double-blind, Double-dummy, Placebo-controlled, Crossover Study to Evaluate the Efficacy of TREXIMET® (Sumatriptan + Naproxen Sodium) vs. Butalbital-containing Combination Medications for the Acute Treatment of Migraine when administered during the Moderate-Severe Migraine Pain, 1 of 2', 'Ankylosing spondylitis', 'naproxen')
('A Crossover, Randomized, Double-blind, Double-dummy, Placebo-controlled Study to Evaluate the Efficacy of TREXIMET®/Sumatriptan + Naproxen Sodium vs. Butalbital-containing Combination Medications for the Acute Treatment of Migraine when administered during Moderate-Severe Migraine Pain, 2 of 2', 'Ankylosing spondylitis', 'naproxen')
('Assessment of the effect of sumatriptan and naproxen sodium combination tablet, sumatriptan tablet, and naproxen sodium tablet treatment on blood pressure when administered intermittently for six months for the acute treatment of migraine attacks, with or without aura, in adults', 'Ankylosing spondylitis', 'naproxen'

In [76]:
connection.commit()